In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import ec

# Generate ECC private key
def generate_ecc_private_key():
    private_key = ec.generate_private_key(ec.SECP192R1(), default_backend())
    return private_key

# Generate ECDH shared key
def generate_ecdh_shared_key(private_key, peer_public_key_bytes):
    peer_public_key = serialization.load_pem_public_key(peer_public_key_bytes, backend=default_backend())
    shared_key = private_key.exchange(ec.ECDH(), peer_public_key)
    return shared_key

# Example usage
# Alice generates her private key
# alice_private_key = generate_ecc_private_key()
# Bob generates his private key
serverPriKey = generate_ecc_private_key()

# # Alice sends her public key to Bob
# alice_public_key_bytes = alice_private_key.public_key().public_bytes(
#     encoding=serialization.Encoding.PEM,
#     format=serialization.PublicFormat.SubjectPublicKeyInfo
# )

# Bob sends his public key to Alice
serverPubKeyBytes = serverPriKey.public_key().public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

# Alice computes the shared key using Bob's public key
# alice_shared_key = generate_ecdh_shared_key(alice_private_key, bob_public_key_bytes)

# Bob computes the shared key using Alice's public key
# bob_shared_key = generate_ecdh_shared_key(bob_private_key, alice_public_key_bytes)

# The shared keys should match
# assert alice_shared_key == bob_shared_key
# print("Shared Key:", alice_shared_key.hex())
randomNo = ''
clientPubKey = ''

In [2]:
Cryptkey = b'\xb9t5RR\x9d\x1d+\xaa\xa3\xd1}\xfb\x1a\x1d\xcb'
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

def encrypt_AES(Cryptkey, data):
    data = bytes.fromhex(data)
    cipher = AES.new(Cryptkey, AES.MODE_CBC)
    ciphertext = cipher.encrypt(pad(data, AES.block_size))
    iv = cipher.iv
    return iv + ciphertext

def decrypt_AES(Cryptkey, encrypted_data):
    encrypted_data = bytes.fromhex(encrypted_data)
    iv = encrypted_data[:AES.block_size]
    ciphertext = encrypted_data[AES.block_size:]
    cipher = AES.new(Cryptkey, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size)
    return plaintext

In [3]:
import numpy as np
def SeedGenerator():
    return get_random_bytes(4).hex()
# SeedGenerator()
# import os
# print(os.urandom(1).hex())
def ValLevel(lvl):
    if len(lvl)==1:
        return '0'+lvl
    else:
        return lvl

In [4]:
def SeedGen(seed):
    global Cryptkey
    print(type(seed))
    return decrypt_AES(Cryptkey,seed).hex()
def ValidateRequest(from_client, step,key=''):
    secAccLvl = int(from_client[3:5])
    global randomNo, clientPubKey
    if from_client[0:2] == '27' and secAccLvl%2==1:
        if step == 0:
            clientPubKey = bytes.fromhex(from_client[6:len(from_client)])
            randomNo = generate_ecdh_shared_key(serverPriKey,clientPubKey)
            seed = SeedGenerator()
            temp = '67 ' + ValLevel(str(secAccLvl)) + ' '+ seed + ' '+ serverPubKeyBytes.hex()
            print(temp)
            return [temp, seed]
        else : 
            print("Wrong request")
            return ['End','']
    if from_client[0:2] == '27' and secAccLvl%2==0 and key!='':
        if step == 1 and +key == SeedGen(from_client[6:len(from_client)]):
            print('67 '+ ValLevel(str(secAccLvl)),'')
            return ['67 '+ ValLevel(str(secAccLvl)),'']        
        else : 
            print("Wrong request/key not matched/empty")
            return ['End','']
    

In [5]:
import socket
import pickle
serv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serv.bind(('0.0.0.0', 8080))
serv.listen(5)
a=0
b =0
while a==0:
  conn, addr = serv.accept()
  from_client = ''
  step = 0
  while b==0:
    data = conn.recv(4096)
    if not data: break
    from_client = data.decode('utf8')
    print (from_client)
    msg = 'ECU Connected'
    conn.send(msg.encode())
    data = conn.recv(4096)
    from_client = data.decode('utf8')
    print (from_client)
    resp = ValidateRequest(from_client,step,'')
    seed = resp[1]
    conn.send(resp[0].encode())
    if resp[0] == 'End':
        a=1
        b=1
        break
    else: step = 1
    data = conn.recv(4096)
    from_client = data.decode('utf8')
    print (from_client)
    resp = ValidateRequest(from_client,step,seed)
    conn.send(resp[0].encode())
    if resp[0] == 'End':
        a=1
        b=1
        break
    else: 
        print("ECU unlocked")
        a = 1
        b =1
        break
    
  conn.close()
print ('terminating connection')

Tester Connected

27 07 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d67414549395a6348474d79785030514d70455a53433661516d50644b7a4c550a34332b7077317037677756454844786f2f6a36562b6c595447375a6272546849597967530a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
67 07 fb463f0e 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741457a4d6c666957636659356c61784a43396769557976727136613753520a344c4f77305367416c6d6e586c716e676134724279423551396e424b4b70783330312f750a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
27 08 a33991aa6d91aeede2b73490f914674882f2f698f57d8ca5c0fb4fcbc9aef384
<class 'str'>
67 08 
ECU unlocked
terminating connection


In [10]:
generate_ecdh_shared_key(serverPriKey,clientPubKey)

b'\x1e^\x8a6~Z\x9b\xc0\xffN\x07\x87\x9aL\xbe\x92\\v4\xc1f\x00\x1b\xd4'

In [11]:
clientPubKey

b'-----BEGIN PUBLIC KEY-----\nMEkwEwYHKoZIzj0CAQYIKoZIzj0DAQEDMgAElAXRRzfn0zU5Ypr/blkQm5STCVe4\n8KVqpPuZEBWN2IktAB2kr0sr+Jn9ku9oCP9M\n-----END PUBLIC KEY-----\n'

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import ec
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import socket
import pickle


def generate_ecc_private_key():
    private_key = ec.generate_private_key(ec.SECP192R1(), default_backend())
    return private_key

def generate_ecdh_shared_key(private_key, peer_public_key_bytes):
    peer_public_key = serialization.load_pem_public_key(peer_public_key_bytes, backend=default_backend())
    shared_key = private_key.exchange(ec.ECDH(), peer_public_key)
    return shared_key

serverPriKey = generate_ecc_private_key()

serverPubKeyBytes = serverPriKey.public_key().public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

randomNo = ''
clientPubKey = ''
Cryptkey = b'\xb9t5RR\x9d\x1d+\xaa\xa3\xd1}\xfb\x1a\x1d\xcb'


def encrypt_AES(Cryptkey, data):
    data = bytes.fromhex(data)
    cipher = AES.new(Cryptkey, AES.MODE_CBC)
    ciphertext = cipher.encrypt(pad(data, AES.block_size))
    iv = cipher.iv
    return iv + ciphertext

def decrypt_AES(Cryptkey, encrypted_data):
    encrypted_data = bytes.fromhex(encrypted_data)
    iv = encrypted_data[:AES.block_size]
    ciphertext = encrypted_data[AES.block_size:]
    cipher = AES.new(Cryptkey, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size)
    return plaintext

def SeedGenerator():
    return get_random_bytes(4).hex()

def ValLevel(lvl):
    if len(lvl)==1:
        return '0'+lvl
    else:
        return lvl
def SeedGen(seed):
    global Cryptkey
    print(type(seed))
    return decrypt_AES(Cryptkey,seed).hex()
def ValidateRequest(from_client, step,key=''):
    secAccLvl = int(from_client[3:5])
    global randomNo, clientPubKey
    if from_client[0:2] == '27' and secAccLvl%2==1:
        if step == 0:
            clientPubKey = bytes.fromhex(from_client[6:len(from_client)])
            randomNo = generate_ecdh_shared_key(serverPriKey,clientPubKey)
            seed = SeedGenerator()
            temp = '67 ' + ValLevel(str(secAccLvl)) + ' '+ seed + ' '+ serverPubKeyBytes.hex()
            print(temp)
            return [temp, seed]
        else : 
            print("Wrong request")
            return ['End','']
    if from_client[0:2] == '27' and secAccLvl%2==0 and key!='':
        if step == 1 and +key == SeedGen(from_client[6:len(from_client)]):
            print('67 '+ ValLevel(str(secAccLvl)),'')
            return ['67 '+ ValLevel(str(secAccLvl)),'']        
        else : 
            print("Wrong request/key not matched/empty")
            return ['End','']

serv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serv.bind(('0.0.0.0', 8080))
serv.listen(5)
a=0
b =0
while a==0:
  conn, addr = serv.accept()
  from_client = ''
  step = 0
  while b==0:
    data = conn.recv(4096)
    if not data: break
    from_client = data.decode('utf8')
    print (from_client)
    msg = 'ECU Connected'
    conn.send(msg.encode())
    data = conn.recv(4096)
    from_client = data.decode('utf8')
    print (from_client)
    resp = ValidateRequest(from_client,step,'')
    seed = resp[1]
    conn.send(resp[0].encode())
    if resp[0] == 'End':
        a=1
        b=1
        break
    else: step = 1
    data = conn.recv(4096)
    from_client = data.decode('utf8')
    print (from_client)
    resp = ValidateRequest(from_client,step,seed)
    conn.send(resp[0].encode())
    if resp[0] == 'End':
        a=1
        b=1
        break
    else: 
        print("ECU unlocked")
        a = 1
        b =1
        break
    
  conn.close()
print ('terminating connection')  